In [1]:
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt

The following is a summary of a research paper: 
Beveridge, Andrew, and Stan Wagon. “The Sorting Hat Goes to college.” Mathematics Magazine, vol. 87, no. 4, Oct. 2014, pp. 243–251, https://doi.org/10.4169/math.mag.87.4.243. 

## Problem 

Assign first-years students to their desired courses (about 500 students must be placed into about 35 first-year courses). Maximize overall student satisfaction such that each student is assigned into a course that they ranked. 

## Solution

### Hungarian Algorithm (minimum weight perfect matching algorithm)

Weighted bipartite graph - n vertices in one part correspond to n vertices in the other part

- create a bipartite graph that enforced the constraints on class size and demographics
- choose a weighting scheme that ensured trade-offs were consistent with the goals an priorities of the AP office

The Hungarian algorithm works well but sometimes the constraints cannot be met, which is why the Integer Linear Programming method works better as it handles a wide variety of constraints.


### Integer Programming problem

**parameters:**
- students $S_i, 1 \leq i \leq n$
- courses $C_j, 1 \leq j \leq m$
- M, F, I are indices corresponding to male, female, and international students
- preferences $P_i$
- $X_j$ : set of students i having $C_j$ as one of their choices, where $j \in P_i$ 
* worst case $X_j = \emptyset$

**Decision Variables**
- $X_{i,j}$ = 1 if for each pair $(S_i, C_j)$ where $j \in P_i$, $S_i$ is assigned to $C_j$, and 0 otherwise
- $\sum_{j \in P_i}$ X_{i,j}$ = 1 for each $i \leq n$ and $x_{i,j} \in \{0,1\}$

**Objective Function**
$$\text{min } \sum_{i=1}^n x_{i,j_{i,1}} + \alpha x_{i,j_{i,2}} + \alpha^2 x_{i,j_{i,3}}  + \alpha^3 x_{i,j_{i,4}}$$ 
where $j_{i,1}, ..., j_{i,4}$ are student $S_i$'s four choices and $\alpha > 1$ 

* $\alpha$ is a value chosen according to the trade-offs for swapping students between classes
The weight of placing a student in a course increases as it is further from the first choice.

**constraints**
1. Each class size should be from 10 to 16 students
2. The demographics of each class should be roughly comparable to the entire student body (60% female and 11% international student)
* sometimes the problem gives infeasible solutions, so the students are placed into an unranked course

**Course size constraint**
- $L \leq \sum_{i \in X_j} x_{i,j} \leq U$ for each $j \leq m$

Typically, we have $(L, U) = (12, 16)$ so the course can have 12 to 16 students. However, it is common to have a course that allows more than 16 or less than 12. We introduce $q_{j,s}$ where s is the size of $C_j$ and $9 \leq s \leq 17$. Suppose we have $Q_{17}, Q_{11}, Q_{10}, Q_9$ that represents the maximum number of students allowed in each class 17, 11, 10, and 9 respectively. Then, the ideal size constraints are:
* $\sum_j q_{j,17} \leq Q_{17}$ 
* $\sum_j q_{j,11} \leq Q_{11}$ 
* $\sum_j q_{j,10} \leq Q_{10}$ 
* $\sum_j q_{j,9} \leq Q_{9}$ 
and $$\sum_{s=9}^{17} q_{j,s} = 1$$
$$\sum_{i \in X_j} x_{i,j} = \sum_{s=9}^{17} sq_{j,s}$$ 


**Gender constraint**

There are at least 4 male and 4 female in each course

* $\sum{i \in M \cap X_j} x_{i,j} \geq 4$
* $\sum{i \in F \cap X_j} x_{i,j} \geq 4$

**International constraint**

Each course contains at most B international students

$\sum_{i \in I \cap X_j} x_{i,j} \leq B$



